In [1]:
import numpy as np
import theano
import theano.tensor as T
import time
from itertools import chain

import lasagne as L

from SimpleRNNLM import SimpleRNNLM, iterate_minibatches

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5005)


In [2]:
# remember, now the pad value is the same as the <utt_end> token

pad_value = -1 # <utt_end>'s vector is the last one

def split_utt(utt):
    u1, u2, u3 = [i for i,j in enumerate(utt) if j == 1]
    return [utt[:u2], utt[u2:u3], utt[u3:]]

In [3]:
mt_path = "/pio/data/data/mtriples/"
mt_path = "/home/maciek/Desktop/mgr/DATA/MovieTriples_Dataset/"

def load_mt(path=mt_path):
    tr = np.load(mt_path + 'Training.triples.pkl')
    vl = np.load(mt_path + 'Validation.triples.pkl')
    ts = np.load(mt_path + 'Test.triples.pkl')
    
#     tr = chain(*map(split_utt, tr))
#     vl = chain(*map(split_utt, vl))
#     ts = chain(*map(split_utt, ts))
    
    return tr, vl, ts

train, valid, test = load_mt()

train = [utt for utt in train if len(utt) < 200]
valid = [utt for utt in valid if len(utt) < 200]
test  = [utt for utt in test  if len(utt) < 200]


def get_mt_voc(mt_path=mt_path, train_len=len(train)):
    word_list = np.load(mt_path + 'Training.dict.pkl')
    word_list.sort(key=lambda x: x[1])
    freqs = np.array(map(lambda x: x[2], word_list) + [train_len])
    total_count = float(sum(freqs))
    
    words = map(lambda x: x[:2], word_list)
    
    w_to_idx = dict(words)
    w_to_idx['<utt_end>'] = pad_value
    idx_to_w = {v : k for (k,v) in w_to_idx.items()}
    
    return idx_to_w, w_to_idx, len(w_to_idx), freqs / total_count

idx_to_w, w_to_idx, voc_size, freqs = get_mt_voc()

In [4]:
word2vec_embs, word2vec_embs_mask = np.load(mt_path + 'Word2Vec_WordEmb.pkl')
word2vec_embs = np.vstack([word2vec_embs, L.init.GlorotUniform()((1,300))]).astype(np.float32)
word2vec_embs_mask = np.vstack([word2vec_embs_mask, np.ones((1,300))])

w2v_train_mask = np.where(word2vec_embs_mask[:,0] == 1)[0]

In [9]:
def update_fn(loss, params):
    return L.updates.adagrad(loss, params, learning_rate=.01)

net = SimpleRNNLM(voc_size=voc_size,
                  emb_size=300,
                  rec_size=300,
                  mode='nce',
                  num_sampled=10,
                  emb_init=word2vec_embs,
                  noise_probs=freqs)

Building the model...
Compiling theano functions...
Building a network for generation...
Done


In [10]:
# net.load_params(fname='5ep_w2vInit_300_300_ssoft(uni,200,non-unique)_bs50_cut200.npz')

In [10]:
net.train_model(num_epochs=1,
                path=None,
                save_params=False,
                train_batch_size=50,
                train_data=train,
                val_batch_size=25,
                val_data=valid)

Done 10 batches in 1.92s	training loss:	29.826126
Done 20 batches in 3.82s	training loss:	28.753693
Done 30 batches in 5.60s	training loss:	28.383765
Done 40 batches in 7.44s	training loss:	27.764355
Done 50 batches in 9.53s	training loss:	26.874535
Done 60 batches in 11.34s	training loss:	26.239686
Done 70 batches in 13.19s	training loss:	25.492427
Done 80 batches in 15.01s	training loss:	25.123941
Done 90 batches in 17.06s	training loss:	24.684413
Done 100 batches in 18.86s	training loss:	24.799316


KeyboardInterrupt: 

In [11]:
def rnd_next_word(probs, size=1):
    return np.random.choice(np.append(np.arange(probs.shape[0]-1), -1).astype(np.int32), 
                            size=size, p=probs)

def beam_search(get_probs_fun, beam=10, init_seq='', mode='rr'):
    utt = map(lambda w: w_to_idx.get(w, w_to_idx['<unk>']), init_seq.split())
    if len(utt) == 0 or utt[0] != 1:
        utt = [1] + utt
    utt = np.asarray(utt, dtype=np.int32)[np.newaxis]
    
    if mode[0] == 's':
        words = get_probs_fun(utt)[0].argpartition(-beam)[-beam:].astype(np.int32)
        words[words==voc_size-1] = pad_value
    elif mode[0] == 'r':
        words = rnd_next_word(get_probs_fun(utt)[0], beam)
    
    candidates = utt.repeat(beam, axis=0)
    candidates = np.hstack([candidates, words[np.newaxis].T])
    scores = np.zeros(beam)
    
#     print candidates
    
    while candidates.shape[1] < 100 and pad_value not in candidates[:,-1]:
        
        if mode[1] == 's':
            log_probs = np.log(get_probs_fun(candidates))
            tot_scores = log_probs + scores[np.newaxis].T

            idx = tot_scores.ravel().argpartition(-beam)[-beam:]
            i,j = divmod(idx, tot_scores.shape[1])
            j[j==voc_size-1] = pad_value
            
            scores = tot_scores[i,j]

            candidates = np.hstack([candidates[i], j[np.newaxis].T.astype(np.int32)])
            
        elif mode[1] == 'r':
            probs = get_probs_fun(candidates)
            words = []
            for k in xrange(beam):
                words.append(rnd_next_word(probs[k], beam)) # this doesn't have to be exactly 'beam'
            words = np.array(words)
            idx = np.indices((beam, words.shape[1]))[0]
            tot_scores = scores[np.newaxis].T + np.log(probs)[idx, words]
                
            idx = tot_scores.ravel().argpartition(-beam)[-beam:]
            i,j = divmod(idx, tot_scores.shape[1])

            scores = tot_scores[i,j]

            candidates = np.hstack([candidates[i], words[i,j][np.newaxis].T])
            
#     print candidates[:,:10]
#     print scores[:10]
        
    cands = candidates[candidates[:,-1] == 0]
    if cands.size > 0:
        return candidates[candidates[:,-1] == 0][0]
    return candidates[scores.argmax()]

In [14]:
utt = beam_search(net.get_probs_fn, init_seq='', beam=2, mode='rr')

text = map(lambda i: idx_to_w[i], list(utt))
' '.join([t for t in text if t not in ['<s>', '</s>', '<utt_end>']])

"really ? i ' m okay . <person> . <person> ' s about to tell me ."

In [100]:
init_seq = ''
utt = [1] + map(lambda w: w_to_idx.get(w, w_to_idx['<unk>']), init_seq.split())
utt = np.asarray(utt, dtype=np.int32)[np.newaxis]

i = 0
while utt[0,-1] != -1 and i < 100:
    word_probs = net.get_probs_fn(utt)[0]
    next_idx = rnd_next_word(word_probs)
    utt = np.append(utt, next_idx)[np.newaxis].astype(np.int32)
    i += 1
    
text = map(lambda i: idx_to_w[i], list(utt[0]))
' '.join([t for t in text if t not in ['<s>', '</s>', '<utt_end>']])

'<person> <unk> ? got some help ! !'